In [3]:
from scapy.all import *

In [4]:
f = rdpcap('../samples/download.pcap')

In [5]:
# Below defines rough implementation

class CoreStructure:
    core_structure = dict()
    raw_scapy_packets = None
    
    def __init__(self, scapy_packets):
        # We are assuming scapy_packets is a perfectly valid scapy packet structure
        self.raw_scapy_packets = scapy_packets
        
    def start(self):
        for packet in self.raw_scapy_packets[:50]:
            l3 = ''
            l4 = ''
            s_ip_addr = None
            d_ip_addr = None
            s_port = None
            d_port = None
            if packet.getlayer(1).name == 'IP':
                l3 = 'IPv4'
                s_ip_addr = packet.getlayer(1).src
                d_ip_addr = packet.getlayer(1).dst
                if packet.getlayer(2).name and packet.getlayer(2).name == 'TCP':
                    l4_packet = packet.getlayer(2) # Storing reference
                    l4 = l4_packet.name
                    s_port = getattr(l4_packet, 'sport')
                    d_port = getattr(l4_packet, 'dport')
                    s_socket = f'{s_ip_addr}:{s_port}'
                    d_socket = f'{d_ip_addr}:{d_port}'
                    # Check if the particular entry exists
                    if not (s_socket, d_socket) in self.core_structure:
                        # Remember src dest will be reversed in packet but
                        # the actual connection is still the same
                        # So we are checking if the reverse tuple exists
                        if not (d_socket, s_socket) in self.core_structure:
                            # If not create a new
                            self.core_structure[(s_socket, d_socket)] = {
                                "l2_proto": packet.name,
                                "l3_proto": l3,
                                "l4_proto": l4_packet.name,
                                "og_seq": getattr(l4_packet, 'seq') ,
                                "up_data": 0,
                                "down_data": 0
                            }
                        else:
                            # This is a reverse scenario, so we will have to swap the src dest
                            # THIS IS A TEMPORARY FIX
                            s_port, d_port = d_port, s_port
                            s_ip_addr, d_ip_addr = d_ip_addr, s_ip_addr
                            s_socket, d_socket = d_socket, s_socket
                    else:
                        current_entry = self.core_structure[(s_socket, d_socket)] # Storing reference
                        print(current_entry)
                        # I am not sure if the below statement will produce correct results
                        # So for now this exists and "kinda" works but shouldn't be assumed final
                        current_entry["up_data"] += getattr(l4_packet, 'seq') - current_entry["og_seq"]

In [6]:
core_struct = CoreStructure(f)

In [7]:
core_struct.start()

{'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'og_seq': 3495802807, 'up_data': 0, 'down_data': 0}
{'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'og_seq': 3495802807, 'up_data': 1, 'down_data': 0}
{'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'og_seq': 442586608, 'up_data': 0, 'down_data': 0}
{'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'og_seq': 3495802807, 'up_data': 2, 'down_data': 0}
{'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'og_seq': 1338928663, 'up_data': 0, 'down_data': 0}
{'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'og_seq': 4045689038, 'up_data': 0, 'down_data': 0}
{'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'og_seq': 1338928663, 'up_data': 1, 'down_data': 0}
{'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'og_seq': 4045689038, 'up_data': 1, 'down_data': 0}
{'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'og_seq':

In [8]:
print(core_struct.core_structure)

{('192.168.0.104:36026', '172.67.216.140:443'): {'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'og_seq': 2540327016, 'up_data': 0, 'down_data': 0}, ('192.168.0.104:54960', '104.17.32.82:443'): {'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'og_seq': 740875834, 'up_data': 0, 'down_data': 0}, ('192.168.0.104:60028', '185.199.110.153:80'): {'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'og_seq': 3495802807, 'up_data': 424, 'down_data': 0}, ('192.168.0.104:60030', '185.199.110.153:80'): {'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'og_seq': 442586608, 'up_data': 1, 'down_data': 0}, ('192.168.0.104:34690', '185.199.108.153:443'): {'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'og_seq': 1338928663, 'up_data': 1556, 'down_data': 0}, ('192.168.0.104:34692', '185.199.108.153:443'): {'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'og_seq': 4045689038, 'up_data': 520, 'down_data': 0}}


## Good enough so far 😄